In [1]:
import copy
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    # source_temp.paint_uniform_color([1, 0.706, 0])
    # target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [3]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [ ]:
def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    # demo_icp_pcds = o3d.data.DemoICPPointClouds()
    # source = o3d.io.read_point_cloud("C:/Users/nguye/Downloads/images/test.ply")
    # target = o3d.io.read_point_cloud("C:/Users/nguye/Downloads/images/test.ply")
    source = o3d.io.read_point_cloud("corner1_voxel.ply")
    target = o3d.io.read_point_cloud("corner2_voxel.ply")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    translation = np.array([1.0, 2.0, 3.0])  # move by x=1.0, y=2.0, z=3.0
    source.translate(translation)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [6]:
voxel_size = 1  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)

:: Load two point clouds and disturb initial pose.
[Open3D WARNING] Read PLY failed: unable to open file: old_data\corner1_voxel.ply
[Open3D WARNING] Read PLY failed: unable to open file: old_data\corner2_voxel.ply
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
:: Downsample with a voxel size 1.000.
:: Estimate normal with search radius 2.000.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
:: Compute FPFH feature with search radius 5.000.


RuntimeError: [Open3D Error] (class std::shared_ptr<class open3d::pipelines::registration::Feature> __cdecl open3d::pipelines::registration::ComputeFPFHFeature(const class open3d::geometry::PointCloud &,const class open3d::geometry::KDTreeSearchParam &)) D:\a\Open3D\Open3D\cpp\open3d\pipelines\registration\Feature.cpp:126: Failed because input point cloud has no normal.


In [67]:
draw_registration_result(source_down, target_down, np.identity(4))

In [68]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 10
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [69]:
draw_registration_result(source, target, result_ransac.transformation)

In [21]:
print(result_ransac.transformation)

[[ -0.10919957   0.80422845   0.58420206   1.5307438 ]
 [  0.35622827  -0.51701801   0.77832756 -13.68526546]
 [  0.92799616   0.29310233  -0.23003077  -0.86478611]
 [  0.           0.           0.           1.        ]]


In [61]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source, target, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 1.000,
   we use a liberal distance threshold 10.000.
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.045475e-15, and correspondence_set size of 3371
Access transformation to get result.


In [45]:
draw_registration_result(source, target, result_ransac.transformation)